In [1]:
#import requests
#from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

import seaborn
from matplotlib import pyplot

from sklearn.linear_model import LogisticRegression

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [2]:
chess_games = pd.read_csv('chess_games.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'chess_games.csv'

In [ ]:
#Data Cleaning - convert date and time columns to datatime objects
chess_games['UTCDate'] = pd.to_datetime(chess_games['UTCDate'])
chess_games['UTCTime'] = pd.to_datetime(chess_games['UTCTime'])

#display(chess_games)
#chess_games['Event'] = chess_games['Event'].astype(str)

#chess_games['ECO'] = chess_games['ECO'].astype('|S')
display(chess_games.head())
print(chess_games.dtypes)

In [ ]:
#checking for unusual values
#%sql SELECT * FROM chess_games WHERE Result = '*'
#%sql SELECT DISTINCT Termination FROM chess_games
#%sql SELECT * FROM chess_games WHERE Termination = 'Rules infraction'

#filter out Termination = Abandoned, Rules infraction, filtering out TimeControl = '-'
#checking that there are no games where result is * (meaning incomplete)
completed_chess_games = %sql SELECT * FROM chess_games \
WHERE Termination NOT IN ('Abandoned', 'Rules Infraction') \
    AND TimeControl != '-' \
    AND Result != '*'


#display(completed_chess_games.head())



#separating TimeControl into 2 columns (base & additional)

In [ ]:
#filter out all NaN values
completed_chess_games = completed_chess_games.dropna()
completed_chess_games.head()

In [ ]:
#separating TimeControl into 2 columns (base & additional)
TOP_completed_chess_games = completed_chess_games.head(10)

In [ ]:
#get the time control split into two columns - Base and Incremental
time_control_df = TOP_completed_chess_games.TimeControl.apply(lambda x: pd.Series(str(x).split("+")))

time_control_df = time_control_df.rename(columns={0:"Base", 1:"Increment"})
time_control_df

In [ ]:
#Join old and new tables together
time_control_split_df = pd.concat([TOP_completed_chess_games, time_control_df], axis=1)
display(time_control_split_df.head())

In [ ]:

#%sql SELECT * FROM chess_games WHERE Result = '*'
%sql SELECT DISTINCT Termination FROM chess_games
#%sql SELECT * FROM chess_games WHERE Termination = 'Rules infraction'
#filter out Termination = Abandoned, Rules infraction, filtering out TimeControl = '-'

#filter out all NaN values

#separating TimeControl into 2 columns (base & additional)

In [ ]:
#Creating a new table for only necessary columns
completed_chess_games = %sql SELECT Event, Result, WhiteElo, BlackElo, (WhiteElo - BlackElo) AS WhiteRatingDiff, ECO, Opening, TimeControl, Termination \
FROM completed_chess_games

In [ ]:
completed_chess_games = completed_chess_games.dropna()
completed_chess_games

In [ ]:
#Summary Statistics and Descriptive Statistics
#%sql top_ten_openings << SELECT Opening, COUNT(White) AS Games_Won FROM chess_games WHERE Result = '1-0' GROUP BY White, Opening ORDER BY Games_Won DESC LIMIT 10
completed_chess_games.describe()

In [ ]:
#Opening distribution - pre slice
opening_dist = completed_chess_games.groupby(by = 'Opening')
opening_dist = opening_dist.count().sort_values(by = 'Event', ascending=False)
#opening_dist

opening_dist_1 = opening_dist.loc[:, ['Event']]
opening_dist_1 = opening_dist_1.rename(columns={"Event":"Count"})
opening_dist_1['Percent_Tot'] = opening_dist_1['Count']/len(completed_chess_games)
opening_dist_1 = opening_dist_1[opening_dist_1['Percent_Tot'] > 0.0001]
#opening_dist_1

#2940 rows - no filter for > 0.01% of Total
#958 rows - filter for > 0.01% of Total


#Sliced distribution
dist_cut = completed_chess_games[completed_chess_games['Event'].index <= len(completed_chess_games)*.10]

#Opening distribution - post slice
opening_dist_cut = dist_cut.groupby(by = 'Opening')
opening_dist_cut = opening_dist_cut.count().sort_values(by = 'Event', ascending=False)
#opening_dist_cut

opening_dist_cut_1 = opening_dist_cut.loc[:, ['Event']]
opening_dist_cut_1 = opening_dist_cut_1.rename(columns={"Event":"CUT_Count"})
opening_dist_cut_1['CUT_Percent_Tot'] = opening_dist_cut_1['CUT_Count']/len(dist_cut)
opening_dist_cut_1 = opening_dist_cut_1[opening_dist_cut_1['CUT_Percent_Tot'] > 0.0001]
#opening_dist_cut_1

#2623 rows - no filter for > 0.01% of Total
#956 rows - filter for > 0.01% of Total


#Joined Tables
open_joined_opening = opening_dist_1.join(opening_dist_cut_1, on = "Opening")
open_joined_opening['Percent_Diff'] = open_joined_opening['Percent_Tot'] - open_joined_opening['CUT_Percent_Tot']
open_joined_opening
#958 rows with NaN included
#924 rows NaN filtered out

In [ ]:
#Time Control distribution - pre slice
timecontrol_dist = completed_chess_games.groupby(by = 'TimeControl')
timecontrol_dist = timecontrol_dist.count().sort_values(by = 'Event', ascending=False)
#timecontrol_dist

timecontrol_dist_1 = timecontrol_dist.loc[:, ['Event']]
timecontrol_dist_1 = timecontrol_dist_1.rename(columns={"Event":"Count"})
timecontrol_dist_1['Percent_Tot'] = timecontrol_dist_1['Count']/len(completed_chess_games)
timecontrol_dist_1 = timecontrol_dist_1[timecontrol_dist_1['Percent_Tot'] > 0.0001]
#timecontrol_dist_1

#840 rows - no filter for 0.01% of Total
#189 rows - filter for 0.01% of Totals


#Sliced distribution
#dist_cut = completed_chess_games[completed_chess_games['Event'].index <= len(completed_chess_games)*.10]

#TimeControl distribution - post slice
timecontrol_dist_cut = dist_cut.groupby(by = 'TimeControl')
timecontrol_dist_cut = timecontrol_dist_cut.count().sort_values(by = 'Event', ascending=False)
timecontrol_dist_cut


timecontrol_dist_cut_1 = timecontrol_dist_cut.loc[:, ['Event']]
timecontrol_dist_cut_1 = timecontrol_dist_cut_1.rename(columns={"Event":"CUT_Count"})
timecontrol_dist_cut_1['CUT_Percent_Tot'] = timecontrol_dist_cut_1['CUT_Count']/len(dist_cut)
timecontrol_dist_cut_1 = timecontrol_dist_cut_1[timecontrol_dist_cut_1['CUT_Percent_Tot'] > 0.0001]
timecontrol_dist_cut_1

#534 rows - no filter for > 0.01% of Total
#182 rows - filter for > 0.01% of Total


#Joined Tables
tc_joined_opening = timecontrol_dist_1.join(timecontrol_dist_cut_1, on = "TimeControl")
tc_joined_opening['Percent_Diff'] = tc_joined_opening['Percent_Tot'] - tc_joined_opening['CUT_Percent_Tot']
tc_joined_opening

